In [1]:
import csv
import numpy as np
import pandas as pd
import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
MINIMUM_UPDATE_INTERVAL = datetime.timedelta(days=1)
src = 'sportsbook_data/fanduel.csv'

In [3]:
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import sleep

In [4]:
import json
with open('config.json', 'r') as file:
    proxy_options = json.load(file)['proxy']

In [5]:
def read_last_line(file_path):
    with open(file_path, mode='r') as file:
        # Using a CSV reader to read the file
        reader = csv.reader(file)
        # Convert reader to list and return the last row
        lines = list(reader)
        return lines[-1]

In [6]:
def calculate_elapsed_time(last_update_time):
    # Assuming last_update_time is a string in ISO 8601 format
    last_update_datetime = datetime.datetime.fromisoformat(last_update_time)
    current_datetime = datetime.datetime.now()
    elapsed_time = current_datetime - last_update_datetime
    return elapsed_time

In [7]:
def starts_with_any(string, substrings):
    for substring in substrings:
        if string.startswith(substring):
            return True
    return False

In [8]:
def update1_src(sel_driver, file_path, url = 'https://sportsbook.fanduel.com/motorsport?tab=formula-1'):
    sel_driver.get(url)
    sel_driver.implicitly_wait(10)

    show_more_buttons = sel_driver.find_elements('xpath', '//div[@aria-label="Show more"]')
    for button in show_more_buttons:
        button.click()

# Grand Prix-specific outright betting may not be available immediately. Hence we relax the condition a bit.
#     tab = sel_driver.find_element('xpath',
#                               '//ul[contains(., " Grand Prix") and contains(., "F1 Outrights")][not(.//ul)]')
    tab = sel_driver.find_element('xpath',
                              '//ul[contains(., "F1 Outrights")][not(.//ul)]')
    
    rates_text = tab.text.split('\n')
    #print(rates_text)
    
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        time = pd.Timestamp.now()
        race = rates_text[0]
        i = 0
        
        if rates_text[1] == 'Outright Betting':
            i += 2
            while (i < len(rates_text)) and not rates_text[i+1].startswith('Show'):
                driver = rates_text[i]
                value = rates_text[i+1]
                try:
                    value = float(value)
                except Exception as e:
                    print("Invalid value:\n", value)
                    break
                new_row = [race, 'win', time, driver, value]
                writer.writerow(new_row)
                i += 2
                
            print("Fanduel outright betting data updated successfully.")
            i += 2
        
        if rates_text[i] == 'F1 Outrights':
            outright_items = ['F1 Drivers Championship','F1 Constructors Championship', 'Betting Without']
            item = ''
            i += 1
            while (i < len(rates_text)) and not rates_text[i].startswith('Overall H2H'):
                if rates_text[i+1].startswith('Show'):
                    i += 2
                    continue
                if starts_with_any(rates_text[i], outright_items):
                    item = rates_text[i]
                    i += 1
                    continue
                driver = rates_text[i]
                value = rates_text[i+1]
                try:
                    value = float(value)
                except Exception as e:
                    print("Invalid value:\n", value)
                    break
                new_row = [race, item, time, driver, value]
                writer.writerow(new_row)
                i += 2
                
            print("Fanduel season-long data updated successfully.")
                
        if rates_text[i].startswith('Overall H2H'):
            while (i+5 < len(rates_text)):
                item = rates_text[i]
                
                driver = rates_text[i+2]
                value = rates_text[i+3]
                try:
                    value = float(value)
                except Exception as e:
                    print("Invalid value:\n", value)
                    break
                new_row = [race, item, time, driver, value]
                writer.writerow(new_row)
                
                driver = rates_text[i+4]
                value = rates_text[i+5]
                try:
                    value = float(value)
                except Exception as e:
                    print("Invalid value:\n", value)
                    break
                new_row = [race, item, time, driver, value]
                writer.writerow(new_row)
                
                i += 7
    
            print('Fanduel H2H data updated successfully.')

In [9]:
# This may not be available immediately after the last Grand Prix finishes
def update2_src(sel_driver, file_path, url = 'https://sportsbook.fanduel.com/motorsport?tab=f1-race-props'):
    try:
        sel_driver.get(url)
        sel_driver.implicitly_wait(10) # Wait for dynamically loaded content
        sel_driver.find_element('xpath', '//div[@aria-label="Show more"]').click()
        tab = sel_driver.find_element('xpath',
                                  '//ul[contains(., " Grand Prix") and contains(., "Podium Finish")][not(.//ul)]')
        podium_texts = tab.text.split('\n')
        race = podium_texts[0]
        podium_texts = podium_texts[2:-2]

    except NoSuchElementException as n:
        print('Cannot find the element. Error message as below:\n', n)

    except Exception as e:
        print("An error occurred:\n", e)
        
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        time = pd.Timestamp.now()
        
        for i in range(len(podium_texts)//2):
            driver = podium_texts[2*i]
            value = podium_texts[2*i+1]
            try:
                value = float(value)
            except Exception as e:
                print("Invalid value:\n", value)
                break
            new_row = [race, 'podium finish', time, driver, value]
            writer.writerow(new_row)

    print("Fanduel podium finish data updated successfully.")
    # pd.read_csv(src).tail()

In [10]:
def update_src_all(file_path):
    # We use sel_driver instead of the usual nomenclature driver to avoid conflict with racing driver
    sel_driver = webdriver.Chrome(seleniumwire_options={'proxy': proxy_options})
    
    # Must use the same driver, or the "options" dictionary would be mutated and reset.
    update1_src(sel_driver, file_path)
    update2_src(sel_driver, file_path)
    
    sel_driver.quit()

In [11]:
test_mode = False

if test_mode:
    src = 'sportsbook_data/fanduel_test.csv'
    update_src_all(src) # forced update

else:
    last_line = read_last_line(src)
    last_update_time = last_line[2]
    elapsed_time = calculate_elapsed_time(last_update_time)

    if elapsed_time >= MINIMUM_UPDATE_INTERVAL:
        update_src_all(src)

    else:
        print("No need to update the CSV files.")

Fanduel outright betting data updated successfully.
Cannot find the element. Error message as below:
 Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@aria-label="Show more"]"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100d66a80 chromedriver + 4385408
1   chromedriver                        0x0000000100d5f38c chromedriver + 4354956
2   chromedriver                        0x000000010097cb0c chromedriver + 281356
3   chromedriver                        0x00000001009bf2f8 chromedriver + 553720
4   chromedriver                        0x00000001009f7d24 chromedriver + 785700
5   chromedriver                        0x00000001009b3eec chromedriver + 507628
6   chromedriver                        0x00000001009b48c4 chromedriver + 510148
7   chromedriver

UnboundLocalError: cannot access local variable 'podium_texts' where it is not associated with a value

Run this in the command line to compile the .py file for task scheduling

jupyter nbconvert --to script proxy_scraper.ipynb

In [16]:
sel_driver = webdriver.Chrome(seleniumwire_options={'proxy': proxy_options})
sel_driver = webdriver.Chrome()
update1_src(sel_driver, 'sportsbook_data/fanduel_test.csv')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//ul[contains(., "F1 Outrights")][not(.//ul)]"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001025bea80 chromedriver + 4385408
1   chromedriver                        0x00000001025b738c chromedriver + 4354956
2   chromedriver                        0x00000001021d4b0c chromedriver + 281356
3   chromedriver                        0x00000001022172f8 chromedriver + 553720
4   chromedriver                        0x000000010224fd24 chromedriver + 785700
5   chromedriver                        0x000000010220beec chromedriver + 507628
6   chromedriver                        0x000000010220c8c4 chromedriver + 510148
7   chromedriver                        0x000000010258643c chromedriver + 41544